In [243]:
import numpy as np
import plotly.express as px

In [244]:
np.random.seed(0)

In [245]:
num_agents = 10_000
threshold_min, threshold_max, threshold_delta = 0., 1., 1e-3

In [246]:
def best_response(x_0, thresholds, priors):
    X = np.arange(x_0, threshold_max, threshold_delta).round(3)
    utilities = np.empty_like(X)
    for i, x in enumerate(X):
        utility = 0.
        for threshold, prior in zip(thresholds, priors):
            utility += prior * (x >= threshold)
        utility -= abs(x - x_0)
        utilities[i] = utility

    return X[np.argmax(utilities)]

In [247]:
def bayesian_update(priors):
    if np.sum(priors) == 0:
        return np.zeros_like(priors)
    return priors / np.sum(priors)

In [ ]:
def balance_priors(priors):
    total = np.sum(priors)
    if total == 1:
        return priors
    indices = priors == 0
    remainder = 1 - total
    p = remainder / indices.sum()
    priors[indices] = p

In [249]:
X = np.random.uniform(size=num_agents)

In [257]:
threshold_true = 0.3
Y_true = X > threshold_true

thresholds = np.arange(threshold_min, threshold_max, 0.1).round(1)

priors = np.zeros_like(thresholds)
priors[4] = 0.3
priors[5] = 0.3
priors[6] = 0.3
priors[7] = 0.01
balance_priors(priors)
assert np.sum(priors) == 1

print(f"thresholds: {thresholds.round(2)}")
print(f"priors    : {priors.round(2)}")

0.015000000000000013 [ True  True  True  True False False False False  True  True]
thresholds: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
priors    : [0.02 0.02 0.02 0.02 0.3  0.3  0.3  0.01 0.02 0.02]


In [ ]:


partitions = [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]]

accuracy_loss = np.empty_like(thresholds)
for i, partition in enumerate(partitions):
    # Partition thresholds and their respective priors
    threshold_partition = thresholds[partition]
    priors_partition = priors[partition]
    # Update priors
    priors_partition = bayesian_update(priors_partition)
    
    # Compute best response
    X_r = np.empty_like(X)
    for xi, x_0 in enumerate(X):
        X_r[xi] = best_response(x_0, threshold_partition, priors_partition)
    # Compute predicted labels
    Y_pred = X_r > threshold_true
    # Compute accuracy loss
    accuracy_loss[i] = np.sum(Y_pred != Y_true) / Y_true.shape[0]

[0.] [0.]
[0.1] [0.]
[0.2] [0.]
[0.3] [0.]
[0.4] [1.]
[0.5] [1.]
[0.6] [1.]
[0.7] [1.]
[0.8] [0.]
[0.9] [0.]


In [215]:
px.line(x=thresholds, y=accuracy_loss, labels={"x": "threshold", "y": "accuracy_loss"}, markers=True)